In [ ]:
#10_04

import cv2
import numpy as np
import matplotlib

#load images
img1 = cv2.imread('right.png')
img2 = cv2.imread('left.png')

#creating a feature extraction
orb = cv2.ORB_create()

#compute the features in both images
kpt1, desc1 = orb.detectAndCompute(img1, None) #img from which we need to extract the features
kpt2, desc2 = orb.detectAndCompute(img2, None) #img from which we need to extract the features

#create matcher and match key points
matcher = cv2.BFMatcher(cv2.NORM_HAMMING)
matches = matcher.knnMatch(desc1, desc2, k=2) #first element of the array matches with other point of the second img

#How can we get rid of false positive
#We can do it by ratio test, if we get second 2 closest pts and we compute ratio, if this ratio is below certain threshold then it is correct

#how to perform the ratio test
good_matches = []
for m,n in matches: #m,n 2 elems in matches pt
    if m.distance < 0.3 * n.distance: #0.8 because to be sure that we get enough pts then we ll lover it
        good_matches.append(m)

#We need to compute homography mtrx because in ideal case we have 2 imgs , they can easily steched together, but
#in real life you introduce new transformation (tranlation, rotation) while shooting, there you cannot use Affine transformation, you need to 
#use prospective transformation. Previously we used to put 4 pts, now we do not need it bevcause we already extracting pts from img automatically we ll simply use the matches
#We need to check if we have at loeast 4 pts for perspective transformation
if len(good_matches) > 4: #then we can computre our homography mtrx
    #convert pts to float 32 othwise opencv complains
    src_points = np.float32([kpt1[m.queryIdx].pt for m in good_matches]) #in order to compute homo mtrx we do not need descriptor we just 
    #need pos pts. We are using good mtch inf=dex to get coordinate of the pt ".pt" in the line above means point. kpt1 does contain other
    #things on top of pts so we specify that we want to extract pts
    #left img is querry img, right img is training img
    #--------------------------------
    dst_points = np.float32([kpt2[m.trainIdx].pt for m in good_matches]) 
    #2 sets of pts are called querryIdx, they belong to left img, and trainIdx is right one. Inside of good_matches we have matches(querryIdx, trainIdx)
    #the position of good matches is stored img array
    #querryIdx contains an index inside of left img, and trainIdx contains an index inside of right img
    #So we will use only good matches, those that passed the ratio test

    #Compute the homography matrix
    M, mask = cv2.findHomography(src_points, dst_points)
    #Now we can apply this set of transformations to see how src pts will fit to dest pts
    #M will contain transformations, rotations
    #transform the img and stitch it together with the right img
    dst = cv2.warpPerspective(img2, M, (img1.shape[1] + img2.shape[1], img1.shape[0])) #why do we need provide the size cz while transformation it can become bigger so to contain all pts
    dst[0:img2.shape[0], 0:img2.shape[1]] = img2.copy()

cv2.namedWindow("Panorama", cv2.WINDOW_KEEPRATIO)
cv2.imshow("Panorama", dst)
cv2.waitKey(0)
    

    

    
        
        



## Natural Language Processing

-Text recognition
-Speech recognition
-Examples
    -Alisa, Siri
-History of NLP
    -Symbolic NLP(1950 - 1990) 

Neural NLP -> NLP with neural network
Reasons why NLP is harder then computer vision:
-imgs contain numbers so with numbers you can do whatever u like
But, nlp was created for humans by humans.
-ambiguity, scale, sparsity, variation, expressivity

Ambiguity -> any language have an ambiguity
ambiguity can be in a different level (sentence is the same but meanings are different)
Nlp does not scale linearly
Proble  of sparcity
there are most frequent words, and less frequent words
Variation
Expressivity -> same meaning can be expressed with different words, expressivity involves also emotions
Sentiment analysis
Analize text found on the web 

Why NLP improved?
Mainly because of hardware power
GPUs are the main component to train the model
Raise of the web

friday we will do nlpk